# Работа с базами данных

Материалы:
* Макрушин С.В. Лекция 6: Работа с базами данных
* https://sqliteonline.com/
* https://docs.python.org/3/library/sqlite3.html
* https://www.geeksforgeeks.org/sql-join-set-1-inner-left-right-and-full-joins/
* https://www.datacamp.com/community/tutorials/group-by-having-clause-sql

## Задачи для совместного разбора

1. Работая с базой данных `Chinook_Sqlite.sqlite`, найдите и выведите на экран имена и фамилии всех заказчиков из Канады

In [1]:
# import sqlite3

# con = sqlite3.connect("./06_database_data/Chinook_Sqlite.sqlite")

# cur = con.cursor()
# # for row in cur.execute("SELECT firstname, lastname FROM customer WHERE country == 'Canada'"):
# #     print(f"- {row[0]} {row[1]}")
# for row in cur.execute("SELECT firstname, lastname FROM customer WHERE country == ?", ('Canada', )):
#     print(f"- {row[0]} {row[1]}")

# cur.close()
# con.close()


2. Найти и вывести на экран названия всех альбомов группы Accept

In [2]:
# cur = con.cursor()
# for i in cur.execute("SELECT title FROM album WHERE artistid in (SELECT artistid FROM artist WHERE name == 'Accept')"):
#     print(i[0])

# cur.close()


3. Создайте базу данных с названием вашей группы. В этой базе данных создайте таблицу Student, содержащую 2 столбца: id и name. Добавьте в таблицу Student информацию о студентах, сидящих с вами по соседству.

## Лабораторная работа 6

1. Создайте файл базы данных c названием `recipes.db`. Создайте объект-курсор. 

In [3]:
import sqlite3
con = sqlite3.connect("./06_database_data/recipes.db")
cur = con.cursor()
# cur.close()
# con.close()

2. Напишите и выполните SQL-запрос для создания таблицы рецептов `Recipe`. Таблица должна содержать следующие поля:
`id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`. Определитесь с типами и составом ключевых полей.

In [4]:
cur.execute('''DROP TABLE IF EXISTS Recipe''')
cur.execute('''
            CREATE TABLE Recipe
            (
                [id] INTEGER PRIMARY KEY, 
                [name] VARCHAR2, 
                [minutes] INTEGER, 
                [submitted] DATETIME, 
                [description] VARCHAR2, 
                [n_ingredients] INTEGER)''')

# cur.close()


3. Напишите и выполните SQL-запрос для создания таблицы отзывов `Review`. Таблица должна содержать следующие поля:
`id`, `user_id`, `recipe_id`, `date`, `rating`, `review`. Определитесь с типами полей, набором ключевых полей. При помощи внешнего ключа соедините две таблицы.

In [5]:
cur.execute('''DROP TABLE IF EXISTS Review''')
cur.execute('''
            CREATE TABLE Review
            (
                [id] INTEGER PRIMARY KEY, 
                [user_id] INTEGER, 
                [recipe_id] INTEGER, 
                [date] DATETIME, 
                [rating] INTEGER, 
                [review] VARCHAR2,
                FOREIGN KEY (user_id) references Recipe (id)
                
            )''')

4. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в созданные таблицы

In [6]:
import csv

with open('../pract2/data/reviews_sample.csv', 'r') as f:
    dr = csv.DictReader(f)
    reviews_to_db = [(i['id'], i['user_id'], i['recipe_id'], i['date'], i['rating'], i['review']) for i in dr]
    # print(reviews_to_db)



with open('../pract5/data/out/recipes_sample_with_tags_ingredients.csv', 'r') as f:
    dr = csv.DictReader(f)
    recipes_to_db = [(i["id"], i["name"], i["minutes"], i["submitted"], i["description"], i["n_ingredients"]) for i in dr]
    # print(recipes_to_db)

cur.executemany("INSERT INTO Review (id, user_id, recipe_id, date, rating, review) VALUES (?, ?, ?, ?, ?, ?);", reviews_to_db)
cur.executemany("INSERT INTO Recipe (id, name, minutes, submitted, description, n_ingredients) VALUES (?, ?, ?, ?, ?, ?);", recipes_to_db)
con.commit()


5. Найдите все рецепты, для выполнения которых нужно ровно 10 ингредиентов. Выведите на экран первые 5 из найденных рецептов.

In [22]:
cur.execute("SELECT * FROM Recipe WHERE n_ingredients == 10").fetchmany(5)

[(246, 'lee s hot crab dip', 45, '1999-09-01', "lee's hot crab dip", 10),
 (289,
  'feijoada  brazilian bean soup  ii',
  150,
  '1999-10-04',
  'brazilian bean soup',
  10),
 (321, 'drunken chicken ii', 65, '1999-10-04', '', 10),
 (383, 'cocoa espresso cooler', 20, '1999-08-30', '', 10),
 (544, 'greek mushroom salad', 38, '1999-08-10', '', 10)]

6. Найдите название рецепта, для выполнения которого требуется больше всего времени.

In [36]:
[i[0] for i in cur.execute("SELECT name FROM Recipe WHERE minutes == (SELECT MAX(minutes) FROM Recipe)")]
    

['strawberry liqueur', 'blueberry liqueur']

7. Запросите у пользователя id рецепта и верните информацию об этом рецепте. Если рецепт отсуствует, выведите соответствующее сообщение.

In [43]:
id_recipe = input("Введите id рецепта: ") # без валидации

flag = False
for i in cur.execute("SELECT * FROM Recipe WHERE id == ?", [id_recipe]):
    flag = True
    print(i)

if not flag:
    print("Recipe not found")

# 236274


(236274, 'strawberry liqueur', 129615, '2007-06-21', 'a beautiful light red shade with a delicious strawberry flavor.', 5)


8. Найдите кол-во отзывов с рейтингом 5.

9. Найдите кол-во уникальных рецептов, не имеющих отзывов с рейтингом, меньше 4. 

10. Найдите кол-во рецептов, опубликованных в 2010 году и имеющих длину не менее 15 минут.

11. Выберите id рецепта, название рецепта, id пользователя, оставившего отзыв, дату отзыва и рейтинг для тех рецептов, которые имеют не менее 3 ингредиентов. Отсортируйте результат по id рецепта.